### Optional Workbook: For Instructional and Take-Home Purposes

Please note that this workbook is provided as an optional resource for further learning and exploration. The analyses and configurations described herein have already been completed and are available for review within Customer Journey Analytics (CJA). This workbook is intended for instructional purposes or as a take-home resource to deepen your understanding of the processes involved.

## Section 1: Initial Setup and Global Variables

In our first section, we prepare our environment for interaction with Adobe Experience Platform (AEP). This involves importing necessary Python libraries, setting up global variables, and loading credentials which will be used throughout the notebook to authenticate and interact with AEP APIs. The necessary configuration steps can be found in our first notebook - `01 query_data.jpynb`

### Key Steps:
- Import libraries essential for making API requests, handling JSON data, and file operations.
- Define global variables to store API keys, client secrets, and other necessary identifiers.
- Load the private key and set up the access token generation function for API authentication.

By the end of this section, our notebook will be equipped to establish a secure connection to AEP.

In [ ]:
import json
import time
import jwt
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

# Load credentials from the previously prepared JSON file
with open('python_config.json', 'r') as file:
    config = json.load(file)

# Set up credentials
api_key = config["client_id"]
client_secret = config["secret"]
org_id = config["org_id"]
technical_account_id = config["tech_id"]
sandbox = config["sandbox"]

# Read the private key from the file path specified in the config
private_key_path = config["pathToKey"]
with open(private_key_path, 'r') as key_file:
    private_key = key_file.read()

# Access Token variable (to be generated)
access_token = None

# Function to generate an access token
def get_access_token():
    global access_token
    expiration = int(time.time()) + 24*60*60  # Token valid for 24 hours
    claim_set = {
        "exp": expiration,
        "iss": org_id,
        "sub": technical_account_id,
        "https://ims-na1.adobelogin.com/s/ent_dataservices_sdk": True,
        "aud": f"https://ims-na1.adobelogin.com/c/{api_key}"
    }
    
    # Generate JWT token
    jwt_token = jwt.encode(claim_set, private_key, algorithm='RS256')

    
    # Exchange JWT token for an access token
    token_response = requests.post(
        "https://ims-na1.adobelogin.com/ims/exchange/jwt/",
        data={
            "client_id": api_key,
            "client_secret": client_secret,
            "jwt_token": jwt_token
        }
    )
    token_response_json = token_response.json()
    if 'access_token' in token_response_json:
        access_token = token_response_json['access_token']
    else:
        raise Exception("Failed to obtain access token")

# Call the function to get the access token
get_access_token()

## Section 2: Creating a Schema and Dataset in Adobe Experience Platform

To leverage Adobe Experience Platform (AEP) for data analysis and customer segmentation, you first need to create a schema that defines the structure of your data. This guide walks you through creating a schema designed for storing propensity scores, cluster IDs, and person IDs.

### Creating a Schema

1. **Navigate to Schemas**: Access the **Schemas** section from the AEP's left navigation panel and click on **Create Schema** in the upper right corner.
2. **Select the Base Class**: Opt for **Individual Profile** as the base class, then click **Next**.
3. **Schema Configuration**: Give your schema a descriptive name, such as "Propensity and Cluster Data", and click **Finish**.
4. **Adding Fields**: Add the following fields to your schema:
   - `clusterid` as Integer under a custom field group if necessary.
   - `propensity` as Double.
   - `personID` as String and mark it as an identity field, linking it to the identifier in use (e.g., ECID).
   - Ensure you **Save** your schema after adding these fields.

### Creating a Dataset from the Schema

1. **Navigate to Datasets**: Find **Datasets** in the left navigation panel and click on **Create Dataset**.
2. **Dataset Creation**: Select **Create dataset from schema**, choose the schema you just created ("Propensity and Cluster Data"), and click **Next**.
3. **Configure the Dataset**: Name your dataset, for instance, "Propensity Scores and Cluster IDs", and finalize by clicking **Finish**.
4. **Retrieving Dataset ID**: Locate your dataset in the list and note the **Dataset ID** on the far right—this ID is essential for programmatically adding data to this dataset.

### Adding the Dataset to Your CJA Connection for Analysis

After creating your dataset in Adobe Experience Platform (AEP), the next crucial step is to incorporate this dataset into Customer Journey Analytics (CJA) for detailed analysis. Follow these steps to make your data actionable in CJA:

1. **Add the Dataset to Your CJA Connection**: Navigate to your CJA connections and select the option to add a new dataset. Choose the dataset you've created, "Propensity Scores and Cluster IDs", ensuring you select the "personID" field as the identifier. It's crucial to select both "backfill historical data" and "add all new data" options to ensure comprehensive data integration.
   
2. **Configure Data in Your CJA Data View**:
   - Open your CJA data view and locate the "Components" section.
   - Navigate to the "Profile Datasets" folder within the components.
   - Add "clusterid" and "propensity" as dimensions to your data view. This allows you to segment and analyze your data based on these attributes.
   - Optionally, you can also add "propensity" as a metric if you plan to perform quantitative analysis on the propensity scores.

By completing these steps, "clusterid" and "propensity scores" are now fully integrated into your CJA environment, ready for use in Analysis Workspace. This enables a deeper understanding of customer behavior and segmentation, leveraging the rich data captured in AEP.

In [ ]:
# NOTE: Due to the time and permissions required, this code serves as an example and will not be executed in this Lab.
# The data upload process has already been completed for you, and the data is ready for exploration in CJA.
# Use this script as a guide for automating AEP data insertion within your own organization in the future.

import requests
import json

# It's a good practice to use variables for IDs and paths
dataset_id = 'YOUR_DATASET_ID'
json_file_path = 'path/to/your/cluster_and_propensities.json'

# Load your AEP credentials and settings from a config file or environment variables for security
with open('config.json', 'r') as config_file:
    config = json.load(config_file)
    org_id = config['org_id']
    sandbox_name = config['sandbox_name']
    access_token = config['access_token']
    api_key = config['api_key']

# Define the headers for your API calls
headers = {
    "Authorization": f"Bearer {access_token}",
    "x-api-key": api_key,
    "x-gw-ims-org-id": org_id,
    "Content-Type": "application/json"
}

# Creating a new batch for the dataset
create_batch_url = f"https://platform.adobe.io/data/foundation/import/batches"
batch_payload = {
    "datasetId": dataset_id,
    "inputFormat": {"format": "json"}
}

# Execute the POST request to create a new batch
batch_response = requests.post(create_batch_url, headers=headers, json=batch_payload)

if batch_response.status_code == 201:
    print("Batch created successfully.")
    batch_id = batch_response.json().get('id')
    
    # Prepare to upload the JSON file to the created batch
    upload_url = f"https://platform.adobe.io/data/foundation/import/batches/{batch_id}/datasets/{dataset_id}/files"
    
    # Change Content-Type header for file upload
    upload_headers = headers.copy()
    upload_headers["Content-Type"] = "application/octet-stream"
    
    # Open the JSON file and read its content
    with open(json_file_path, 'rb') as file:
        file_content = file.read()
        
        # Execute the PUT request to upload the file
        upload_response = requests.put(upload_url, headers=upload_headers, data=file_content)
        
        if upload_response.status_code in [200, 201, 202]:
            print("JSON file uploaded successfully.")
        else:
            print("Failed to upload JSON file.", upload_response.status_code, upload_response.text)
else:
    print("Failed to create batch.", batch_response.status_code, batch_response.text)

## Section 3: Reviewing Results in Analysis Workspace

With the data now structured and uploaded into Adobe Experience Platform (AEP), you can dive into the insights it offers in Analysis Workspace. We've set up a project that brings the data to life through visualization.

To access the project, click the following link:

[View Analysis Workspace Project](https://experience.adobe.com/#/@acxpevangelist/platform/analytics/#/workspace/edit/65da9110c444a601f2ff8ac6)

Use these credentials (Company or School Account):
- User: 'REDACTED CREDENTIALS'
- Password: 'REDACTED CREDENTIALS'

Upon entering the project, consider following these steps for a guided exploration:

1. **Examine the Propensity Score Distribution**: Start with the histogram to understand the spread of propensity scores across customers.

2. **Identify Key Products**: Look at the 'Products Most Viewed by High Propensity Visitors' to pinpoint items that are drawing in potential buyers.

3. **Spot Top Entry Points**: Use the 'Most Common Entry Pages for High Propensity Visitors' chart to see where these valuable customers begin their journey.

4. **Analyze Referral Sources**: Check the 'Most Common Referrers for High Propensity Visitors' to understand where your high-value traffic is coming from.

5. **Explore Cluster Behaviors**: Delve into the 'Cluster ID Summary Metrics' to dissect the characteristics and behaviors of each customer segment.

6. **Activate Audiences**: Use the audience activation section to create segments for targeted marketing actions.

Remember, this workspace is interactive. Filter, slice, and drill down into the data to uncover actionable insights tailored to your business objectives!